In [1]:
import numpy as np
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

model = TFBertForSequenceClassification.from_pretrained("/Users/dan/Documents/Coding/Audit_LLM/llm_models/fine-tuned-aaa_type_bioclinicalbert")
tokenizer = BertTokenizer.from_pretrained("/Users/dan/Documents/Coding/Audit_LLM/llm_models/fine-tuned-aaa_type_bioclinicalbert")

2024-08-24 21:43:08.199645: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-08-24 21:43:08.199668: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-24 21:43:08.199679: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-24 21:43:08.199695: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-24 21:43:08.199706: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Some layers from the model checkpoint at /Users/dan/Documents/Coding/Audit_LLM/llm_models/fine-tuned-aaa_type_bioclinicalbert were not used when initializing TFBertForSequenceClassif

In [2]:

def sliding_window_tokenize(text, tokenizer, max_length, stride):
    tokens = tokenizer.tokenize(text)
    segments = []

    for i in range(0, len(tokens), stride):
        segment = tokens[i:i + max_length]
        if len(segment) < max_length:
            segment = segment + ['[PAD]'] * (max_length - len(segment))
        segments.append(segment)

        if len(segment) < max_length:
            break

    print(f"Generated {len(segments)} segments.")
    return segments

def tokens_to_inputs(tokens, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    for segment in tokens:
        inputs = tokenizer.encode_plus(
            segment,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )
        input_ids.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])

    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)

    return input_ids, attention_masks

def predict_probabilities_with_aggregation(text, tokenizer, model, max_length=510, stride=255, batch_size=8):
    tokenized_segments = sliding_window_tokenize(text, tokenizer, max_length, stride)
    input_ids, attention_masks = tokens_to_inputs(tokenized_segments, tokenizer, max_length)

    num_batches = (input_ids.shape[0] + batch_size - 1) // batch_size
    segment_probabilities = []

    for i in range(num_batches):
        batch_input_ids = input_ids[i * batch_size:(i + 1) * batch_size]
        batch_attention_masks = attention_masks[i * batch_size:(i + 1) * batch_size]

        outputs = model(batch_input_ids, attention_mask=batch_attention_masks)
        probabilities = tf.nn.softmax(outputs.logits, axis=-1).numpy()
        segment_probabilities.extend(probabilities)

    avg_probabilities = np.mean(segment_probabilities, axis=0)
    return avg_probabilities

def apply_threshold(probabilities):
    print(f"Probabilities: {probabilities}")
    return np.argmax(probabilities)

label_mapping = {
    0: "Non-AAA",
    1: "Primary AAA Repair",
    2: "Revision AAA Repair"
}

def classify_text(text, tokenizer, model):
    probabilities = predict_probabilities_with_aggregation(text, tokenizer, model)
    predicted_label_idx = apply_threshold(probabilities)
    classification = label_mapping[predicted_label_idx]
    return classification, probabilities[predicted_label_idx]

In [5]:
# Example clinical note input
text = " \nName: Douglas Ward                   Unit No: 478 214 277\n \nAdmission Date: 2024-06-02              Discharge Date: 2024-06-12\n \nDate of Birth: 1944-12-10             Sex:   M\n \nService: SURGERY\n \nAllergies: \nPenicillins\n \nAttending: Dr. Jane Johnson.\n \nChief Complaint:\nAbdominal Aortic Aneurysm s/p EVAR repair with type IA endoleak \nrequiring repair \n \nMajor Surgical or Invasive Procedure:\n___ - Fenestrated endovascular aortic repair with a \nphysician modified ___/ 4 visceral endoprostheses\n\n \nHistory of Present Illness:\nThis is an ___ gentleman who had previously undergone an \nEVAR for an infrarenal abdominal aortic aneurysm with a Gore \nExcluder graft.  He subsequently was seen to have aneurysm sac \nexpansion, thought to be secondary to a type 2 endoleak and thus \nunderwent coil embolization of his inferior mesenteric artery as \nwell as several lumbar vessels.  Unfortunately, his sac \ncontinued to expand.  It was thought that his graft was losing \nproximal fixation and an additional cuff was placed proximally \nas well. However, he still had persistent sac expansion and it \nwas determined to be likely due to a type IA endoleak.  He was \nthus consented for placement of a physician-modified fenestrated \nproximal component to extend his proximal field of his repair.\n \nPast Medical History:\nBilateral carotid artery stenosis, coronary artery disease, \nchronic obstructive pulmonary disease, insulin-dependent \ndiabetes mellitus, vasovagal syncope. \n\nPSH: SFA pseudoaneurysm repair with stent in ___, multiple \ncardiac catheterizations ___ and ___, right knee \nreplacement, inguinal hernia repair and multiple minor skin \nprocedures, prior EVAR\n\n \nSocial History:\n___\nFamily History:\nFather had back pain in old age\n \nPhysical Exam:\nDischarge Physical Exam: \nVitals: AVSS, see flowsheets\nGEN: No distress, pleasant, conversant\nHEENT:  Sclera non-icteric, neck is supple without \nlymphadenopathy, thyromegaly or JVD\nHEART:  RRR with no murmurs\nCHEST:  No increased work of breathing, clear to auscultation \nbilaterally, no crackles or wheezes\nABDOMEN: Soft, non-tender, no rebound or guarding\nINCISIONS: Incisions are clean, dry and intact without erythema \npr swelling\nEXTREMITIES: Warm, well perfused, no edema, ___ pulses palpable \nbilaterally. \n \nPertinent Results:\n___ 05:35AM BLOOD WBC-6.6 RBC-3.02* Hgb-8.4* Hct-27.1* \nMCV-90 MCH-27.8 MCHC-31.0* RDW-14.9 RDWSD-48.6* Plt ___\n___ 01:43AM BLOOD WBC-8.1 RBC-3.09* Hgb-8.7* Hct-27.5* \nMCV-89 MCH-28.2 MCHC-31.6* RDW-14.7 RDWSD-47.1* Plt ___\n___ 03:35PM BLOOD WBC-7.6 RBC-3.07* Hgb-8.6* Hct-27.3* \nMCV-89 MCH-28.0 MCHC-31.5* RDW-14.6 RDWSD-46.3 Plt ___\n___ 05:28AM BLOOD WBC-7.0 RBC-3.02* Hgb-8.8* Hct-27.1* \nMCV-90 MCH-29.1 MCHC-32.5 RDW-14.6 RDWSD-47.6* Plt ___\n___ 02:52PM BLOOD WBC-7.0 RBC-3.27* Hgb-9.3* Hct-29.4* \nMCV-90 MCH-28.4 MCHC-31.6* RDW-14.6 RDWSD-48.1* Plt ___\n___ 05:35AM BLOOD Plt ___\n___ 05:35AM BLOOD ___ PTT-33.1 ___\n___ 01:43AM BLOOD Plt ___\n___ 01:43AM BLOOD ___ PTT-32.9 ___\n___ 05:35AM BLOOD Glucose-94 UreaN-23* Creat-1.4* Na-136 \nK-4.4 Cl-102 HCO3-25 AnGap-9*\n___ 01:43AM BLOOD Glucose-114* UreaN-20 Creat-1.2 Na-134* \nK-4.7 Cl-101 HCO3-23 AnGap-10\n___ 03:35PM BLOOD Glucose-128* UreaN-22* Creat-1.2 Na-135 \nK-4.3 Cl-99 HCO3-24 AnGap-12\n___ 01:43AM BLOOD ALT-10 AST-22 LD(___)-333* AlkPhos-93 \nAmylase-26 TotBili-0.5\n___ 02:52PM BLOOD ALT-8 AST-21 LD(LDH)-247 AlkPhos-97 \nAmylase-35 TotBili-0.6\n___ 05:35AM BLOOD Calcium-8.0* Phos-2.5* Mg-2.0\n___ 01:43AM BLOOD Albumin-2.7* Calcium-8.1* Phos-2.9 Mg-2.___ is a ___ year-old man who was admitted to the ___ \n___ on ___. The patient was \ntaken to the endovascular suite and underwent a \nphysician-modified fenestrated ___ for repair of type 1A \nendoleak. For details of the procedure, please see the surgeon's \noperative note. The patient tolerated the procedure well without \ncomplications and was brought to the post-anesthesia care unit \nin stable condition. \n\nAfter a brief stay, the patient was transferred to the \ncardiovascular intensive care unit for close monitoring and for \nmonitoring of his lumbar drain. On POD#1, the lumbar drain was \nremoved without complications. On POD#1 the patient was noted to \nexperience nausea and vomiting and remained in the ICU for an \nadditional day. On POD#2 the patient was transferred to the \nvascular surgery floor where he remained through the rest of the \nhospitalization.\n\nPost-operatively, he did well without any groin swelling. He was \nable to tolerate a regular diet, get out of bed and ambulate \nwithout assistance, void without issues, and pain was controlled \non oral medications alone. He was deemed ready for discharge on \n___, and was given the appropriate discharge and follow-up \ninstructions.\n \nMedications on Admission:\nThe Preadmission Medication list is accurate and complete.\n1. Acetaminophen 650 mg PO Q6H:PRN Pain - Mild \n2. Mirtazapine 30 mg PO QHS \n3. Allopurinol ___ mg PO DAILY \n4. Atenolol 25 mg PO DAILY \n5. Omeprazole 20 mg PO DAILY \n6. Vitamin B Complex 1 CAP PO DAILY \n7. Vitamin D ___ UNIT PO DAILY \n\n \nDischarge Medications:\n1.  Aspirin EC 81 mg PO DAILY  \n2.  Atorvastatin 40 mg PO QPM \nRX *atorvastatin 40 mg 1 tablet(s) by mouth Daily Disp #*30 \nTablet Refills:*6 \n3.  Docusate Sodium 100 mg PO BID  \n4.  Senna 8.6 mg PO BID:PRN constipation  \n5.  Acetaminophen 650 mg PO Q6H:PRN Pain - Mild  \n6.  Allopurinol ___ mg PO DAILY  \n7.  Atenolol 25 mg PO DAILY  \n8.  Mirtazapine 30 mg PO QHS  \n9.  Omeprazole 20 mg PO DAILY  \n10.  Vitamin B Complex 1 CAP PO DAILY  \n11.  Vitamin D ___ UNIT PO DAILY  \n\n \nDischarge Disposition:\nHome With Service\n \nFacility:\n___\n \n___ Diagnosis:\n___ - endoleak s/p EVAR ___ for AAA\n\n \nDischarge Condition:\nMental Status: Clear and coherent.\nLevel of Consciousness: Alert and interactive.\nActivity Status: Ambulatory - Independent.\n\n \nDischarge Instructions:\nDear ___,\n\nYou were admitted to ___ and \nunderwent a fenestrated endovascular aorta repair. You have now \nrecovered from surgery and are ready to be discharged. Please \nfollow the instructions below to continue your recovery:\n\nMEDICATIONS:\n\u2022 Take Aspirin 325mg (enteric coated) once daily \n\u2022 Do not stop Aspirin unless your Vascular Surgeon instructs you \nto do so. \n\u2022 Continue all other medications you were taking before surgery, \nunless otherwise directed\n\u2022 You make take Tylenol or prescribed pain medications for any \npost procedure pain or discomfort\n\nWHAT TO EXPECT AT HOME:\n It is normal to have slight swelling of the legs:\n\u2022 Elevate your leg above the level of your heart (use ___ \npillows or a recliner) every ___ hours throughout the day and at \nnight\n\u2022 Avoid prolonged periods of standing or sitting without your \nlegs elevated\n It is normal to feel tired and have a decreased appetite, your \nappetite will return with time \n\u2022 Drink plenty of fluids and eat small frequent meals\n\u2022 It is important to eat nutritious food options (high fiber, \nlean meats, vegetables/fruits, low fat, low cholesterol) to \nmaintain your strength and assist in wound healing\n\u2022 To avoid constipation: eat a high fiber diet and use stool \nsoftener while taking pain medication\n\nACTIVITIES:\n\u2022 When you go home, you may walk and go up and down stairs\n\u2022 You may shower (let the soapy water run over groin incision, \nrinse and pat dry)\n\u2022 Your incision may be left uncovered, unless you have small \namounts of drainage from the wound, then place a dry dressing or \nband aid over the area that is draining, as needed\n\u2022 No heavy lifting, pushing or pulling (greater than 5 lbs) for \n1 week (to allow groin puncture to heal)\n\u2022 After 1 week, you may resume sexual activity\n\u2022 After 1 week, gradually increase your activities and distance \nwalked as you can tolerate\n\u2022 No driving until you are no longer taking pain medications\n\nCALL THE OFFICE FOR: ___\n\u2022 Numbness, coldness or pain in lower extremities \n\u2022 Temperature greater than 101.5F for 24 hours\n\u2022 New or increased drainage from incision or white, yellow or \ngreen drainage from incisions\n\u2022 Bleeding from groin puncture site\n\nFOR SUDDEN, SEVERE BLEEDING OR SWELLING (Groin puncture site or \nincision)\n\u2022 Lie down, keep leg straight and have someone apply firm \npressure to area for 10 minutes. If bleeding stops, call \nvascular office. If bleeding does not stop, call ___ for \ntransfer to closest Emergency Room. \n\nSincerely, \n\nYour ___ Vascular Surgery Team\n \nFollowup Instructions:\n___\n"

# Prediction using the sliding window approach
probabilities = predict_probabilities_with_aggregation(text, tokenizer, model)
predicted_label_idx = apply_threshold(probabilities)
predicted_label = label_mapping[predicted_label_idx]

# Print the predicted label
print("Predicted Label:", predicted_label)

Generated 10 segments.
Probabilities: [0.0595326  0.26012158 0.6803459 ]
Predicted Label: Revision AAA Repair
